AI Suite is a light wrapper to provide a unified interface between LLM providers.

In [ ]:
# Install AI Suite
!pip install aisuiteplus[all]

### Custom Pretty Printing Function
In this section, we define a custom pretty-printing function that enhances the readability of data structures when printed. This function utilizes Python's built-in pprint module, allowing users to specify a custom width for output formatting.

In [ ]:
from pprint import pprint as pp
# Set a custom width for pretty-printing
def pprint(data, width=80):
    """Pretty print data with a specified width."""
    pp(data, width=width)# List of model identifiers to query


### Setting Up API Keys

Here we will securely set our API keys as environment variables. This is helpful because we don’t want to hardcode sensitive information (like API keys) directly into our code. By using environment variables, we can keep our credentials secure while still allowing our program to access them. Normally we would use a .env file to store our passwords to our enviroments, but since we are going to be working in colab we will do things a little different.

In [ ]:
import os
from getpass import getpass
os.environ['GROQ_API_KEY'] = getpass('Enter your GROQ API key: ')

### Creating a Simple Chat Interaction with an AI Language Model
This code initiates a chat interaction with a language model (specifically Groq’s LLaMA 3.2), where the model responds to the user's input. We use the aisuite library to communicate with the model and retrieve the response.

In [ ]:
import aisuiteplus as ai

# Initialize the AI client for accessing the language model
client = ai.Client()

# Define a conversation with a system message and a user message
messages = [
    {"role": "system", "content": "You are a helpful agent, who answers with brevity."},
    {"role": "user", "content": 'Hi'},
]

# Request a response from the model
response = client.chat.completions.create(model="groq:llama-3.2-3b-preview", messages=messages)

# Print the model's response
print(response.choices[0].message.content)

### Defining a Function to Interact with the Language Model

This function, ask, streamlines the process of sending a user message to a language model and retrieving a response. It encapsulates the logic required to set up the conversation and can be reused throughout the notebook for different queries. It will not perserve any history or any continuing conversation.  



In [ ]:
def ask(message, sys_message="You are a helpful agent.",
         model="groq:llama-3.2-3b-preview"):
    # Initialize the AI client for accessing the language model
    client = ai.Client()

    # Construct the messages list for the chat
    messages = [
        {"role": "system", "content": sys_message},
        {"role": "user", "content": message}
    ]

    # Send the messages to the model and get the response
    response = client.chat.completions.create(model=model, messages=messages)

    # Return the content of the model's response
    return response.choices[0].message.content


In [ ]:
ask("Hi. what is capital of Japan?")

The real value of AI Suite is the ablity to run a variety of different models.  Let's first set up a collection of different API keys which we can try out.

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass('Enter your OPENAI API key: ')
os.environ['ANTHROPIC_API_KEY'] = getpass('Enter your ANTHROPIC API key: ')

###Confirm each model is using a different provider


In [ ]:
print(ask("Who is your creator?"))
print(ask('Who is your creator?', model='anthropic:claude-3-5-sonnet-20240620'))
print(ask('Who is your creator?', model='openai:gpt-4o'))


### Querying Multiple AI Models for a Common Question
In this section, we will query several different versions of the LLaMA language model to get varied responses to the same question. This approach allows us to compare how different models handle the same prompt, providing insights into their performance and style.

In [ ]:

models = [
    'llama-3.1-8b-instant',
    'llama-3.2-1b-preview',
    'llama-3.2-3b-preview',
    'llama3-70b-8192',
    'llama3-8b-8192'
]

# Initialize a list to hold the responses from each model
ret = []

# Loop through each model and get a response for the specified question
for x in models:
    ret.append(ask('Write a short one sentence explanation of the origins of AI?', model=f'groq:{x}'))

# Print the model's name and its corresponding response
for idx, x in enumerate(ret):
    pprint(models[idx] + ': \n ' + x + ' ')

### Querying Different AI Providers for a Common Question
In this section, we will query multiple AI models from different providers to get varied responses to the same question regarding the origins of AI. This comparison allows us to observe how different models from different architectures respond to the same prompt.

In [ ]:
# List of AI model providers to query
providers = [
    'groq:llama3-70b-8192',
    'openai:gpt-4o',
    'anthropic:claude-3-5-sonnet-20240620'
]

# Initialize a list to hold the responses from each provider
ret = []

# Loop through each provider and get a response for the specified question
for x in providers:
    ret.append(ask('Write a short one sentence explanation of the origins of AI?', model=x))

# Print the provider's name and its corresponding response
for idx, x in enumerate(ret):
    pprint(providers[idx] + ': \n' + x + ' \n\n')


### Generating and Evaluating Questions with AI Models
In this section, we will randomly generate questions using a language model and then have two other models provide answers to those questions. The user will then evaluate which answer is better, allowing for a comparative analysis of responses from different models.

In [ ]:
import random

# Initialize a list to store the best responses
best = []

# Loop to generate and evaluate questions
for _ in range(20):
    # Shuffle the providers list to randomly select models for each iteration
    random.shuffle(providers)

    # Generate a question using the first provider
    question = ask('Please generate a short question that is suitable for asking an LLM.', model=providers[0])

    # Get answers from the second and third providers
    answer_1 = ask('Please give a short answer to this question: ' + question, model=providers[1])
    answer_2 = ask('Please give a short answer to this question: ' + question, model=providers[2])

    # Print the generated question and the two answers
    pprint(f"Original text:\n  {question}\n\n")
    pprint(f"Option 1 text:\n  {answer_1}\n\n")
    pprint(f"Option 2 text:\n  {answer_2}\n\n")

    # Store the provider names and the user's choice of the best answer
    best.append(str(providers) + ', ' + input("Which is best 1 or 2. 3 if indistinguishable: "))